###  Module & Utility Imports

In [1]:
import pandas as pd
import numpy as np

### Comparison between the main model and benchmark models (Main manuscript)

In [2]:
Prefix_map = { 'ConVAE' : 'C-VAE',
               'FACVAE' : 'FAC-VAE',
              'TCVAE' : 'TC-VAE',
              'VDVAE' : 'VD-VAE'}


# Load original dataframe
BenchAnalAccMItable = pd.read_csv('./EvalResults/SummaryTables/BenchAnalAccMItable.csv')
BenchAnalAccMItable['Type'] = BenchAnalAccMItable['Type'].replace({'ART': 'ABP', 'II': 'ECG'})

# Pivot table with 'ISCOREScal' values
PivotTable = BenchAnalAccMItable.pivot_table(
    index=['Source', 'Type'],
    columns='Prefix',
    values='ISCOREScal'
).reset_index()

# Rename 'SKZFC' to 'Ours' and move to last column
PivotTable = PivotTable.rename(columns={'SKZFC': 'Ours'})
cols = [col for col in PivotTable.columns if col not in ['Ours']] + ['Ours']
PivotTable = PivotTable[cols]
#PivotTable = np.round(PivotTable, 3)

# Identify model columns (exclude 'Source' and 'Type')
model_cols = [col for col in PivotTable.columns if col not in ['Source', 'Type']]

# Bold the max ISCOREScal in each row
def bold_max_in_row(row):
    values = row[model_cols].astype(float)
    max_val = values.max()
    return [
        f"\\textbf{{{v:.3f}}}" if np.isclose(v, max_val) else f"{v:.3f}"
        for v in values
    ]

# Apply bold formatting row-wise
formatted_model_values = PivotTable[model_cols].apply(lambda row: bold_max_in_row(row), axis=1, result_type='expand')
formatted_model_values.columns = model_cols

# Combine with Source and Type
formatted_table = pd.concat([PivotTable[['Source', 'Type']], formatted_model_values], axis=1)
formatted_table = formatted_table.rename(columns=Prefix_map).copy()

# Generate LaTeX
latex_table_code = r"""\begin{table}[t]
\centering
\caption{Table of benchmark comparisons focused on ISCORE only.}
\label{tab:iscore_comparison}
""" + formatted_table.to_latex(
    index=False,
    escape=False,
    float_format="%.4f",
    column_format="ll|" + "r"*(len(formatted_table.columns)-2),
    bold_rows=False
) + r"""\end{table}"""

# Output LaTeX
print(latex_table_code)


\begin{table}[t]
\centering
\caption{Table of benchmark comparisons focused on ISCORE only.}
\label{tab:iscore_comparison}
\begin{tabular}{ll|rrrrrrrr}
\toprule
Source & Type & C-VAE & DiffWave & FAC-VAE & TC-VAE & VD-VAE & VDWave & Wavenet & Ours \\
\midrule
Mimic & ABP & 0.203 & 0.334 & 0.196 & 0.120 & 0.130 & 0.334 & 0.236 & \textbf{2.083} \\
Mimic & ECG & 0.565 & 0.637 & 0.243 & 0.584 & 0.255 & 0.637 & 0.258 & \textbf{2.442} \\
VitalDB & ABP & 0.153 & 0.342 & 0.160 & 0.187 & 0.137 & 0.342 & 0.180 & \textbf{1.957} \\
VitalDB & ECG & 0.440 & 0.658 & 0.400 & 0.565 & 0.336 & 0.659 & 0.472 & \textbf{2.018} \\
\bottomrule
\end{tabular}
\end{table}


### Comparison between the main model and benchmark models (Supplementary file)

In [23]:
# Check the actual column names
actual_columns = BenchAnalAccMItable.columns.tolist()

# Define major metrics based on actual column names
major_metrics = [
    '(i) $I(V; \\acute{Z} \\mid Z)$',
    '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$',
    '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$',
    'FQI', 'RMSE', 'ISCOREScal'
]

type_name_map = {
    'ART': 'ABP',
    'II': 'ECG'
}


# Get unique values for Source and Type columns
sources = BenchAnalAccMItable['Source'].unique()
types = BenchAnalAccMItable['Type'].unique()

# Rename 'SKZFC' to 'Ours' for easier interpretation
BenchAnalAccMItable.loc[BenchAnalAccMItable['Prefix'] == 'SKZFC', 'Prefix'] = 'Ours'

# Initialize a dictionary to store the generated LaTeX tables
split_tables_latex = {}

# Generate LaTeX table for each (Source, Type) pair
for source in sources:
    for type_ in types:
        subset = BenchAnalAccMItable[
            (BenchAnalAccMItable['Source'] == source) & 
            (BenchAnalAccMItable['Type'] == type_)
        ]
        
        table = subset.pivot_table(index='Prefix', values=major_metrics)
        table = table.reset_index()
        table = table[['Prefix'] + major_metrics].copy()
        
        # Rename columns
        col_map = {
            'Prefix': 'Model', 
            '(i) $I(V; \\acute{Z} \\mid Z)$': '(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$',
            '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$': '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$',
            '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$': '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$', 
            'FQI': 'FQI $\\downarrow$',
            'RMSE': 'RMSE $\\downarrow$',
            'ISCOREScal': 'ISCORE $\\uparrow$'
        }
        table = table.rename(columns=col_map)
        
        # Move Ours to last
        if 'Ours' in table['Model'].values:
            ours_row = table[table['Model'] == 'Ours']
            table = table[table['Model'] != 'Ours']
            table = pd.concat([table, ours_row], ignore_index=True)

        # Bold max values for ↑ metrics and min values for ↓ metrics
        metrics_up = ['(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$',
                      '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$',
                      'ISCORE $\\uparrow$']
        metrics_down = ['(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$',
                        'FQI $\\downarrow$', 'RMSE $\\downarrow$']

        for col in metrics_up:
            max_val = table[col].max()
            table[col] = table[col].apply(lambda x: f"\\textbf{{{x:.3f}}}" if np.isclose(x, max_val) else f"{x:.3f}")

        for col in metrics_down:
            min_val = table[col].min()
            table[col] = table[col].apply(lambda x: f"\\textbf{{{x:.3f}}}" if np.isclose(x, min_val) else f"{x:.3f}")
        
        table = table.rename(columns={'(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$' : '$MS \\uparrow$' , 
             '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$' : '$AC \\uparrow$' , 
             '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$' : '$MP \\downarrow$'})
        
        # Generate LaTeX
        latex_code = r"""\begin{table}[h]
\centering
\caption{Benchmark comparisons for """ + f"{type_name_map.get(type_, type_)}" + r""" on the """ + f"{source}" + r"""}
\label{tab:result_""" + f"{source.lower()}_{type_.lower()}" + r"""}
""" + table.to_latex(index=False, escape=False, column_format="c|" + "c"*(len(table.columns)-2)+'|c' )  + r"""\end{table}"""

        split_tables_latex[(source, type_)] = latex_code


# Example: print one latex table
print(split_tables_latex[('Mimic', 'ABP')])


\begin{table}[h]
\centering
\caption{Benchmark comparisons for ABP on the Mimic}
\label{tab:result_mimic_abp}
\begin{tabular}{c|ccccc|c}
\toprule
Model & $MS \uparrow$ & $MP \downarrow$ & $AC \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
ConVAE & 0.052 & 0.934 & 0.250 & 0.098 & 3.115 & 0.203 \\
DiffWave & 0.000 & 0.002 & 0.001 & 0.044 & 5.027 & 0.334 \\
FACVAE & 0.061 & 0.960 & 0.206 & 0.088 & 2.936 & 0.196 \\
TCVAE & 0.129 & 1.597 & 0.236 & 0.109 & 3.244 & 0.120 \\
VDVAE & 0.066 & 1.358 & 0.112 & 0.089 & \textbf{2.856} & 0.130 \\
VDWave & 0.000 & \textbf{0.001} & 0.000 & \textbf{0.044} & 5.009 & 0.334 \\
Wavenet & 0.000 & 0.475 & 0.049 & 0.075 & 4.617 & 0.236 \\
Ours & \textbf{3.630} & 0.048 & \textbf{0.836} & 0.088 & 3.689 & \textbf{2.083} \\
\bottomrule
\end{tabular}
\end{table}


In [24]:
print(split_tables_latex[('Mimic', 'ABP')])
print()
print(split_tables_latex[('Mimic', 'ECG')])
print()
print(split_tables_latex[('VitalDB', 'ABP')])
print()
print(split_tables_latex[('VitalDB', 'ECG')])

\begin{table}[h]
\centering
\caption{Benchmark comparisons for ABP on the Mimic}
\label{tab:result_mimic_abp}
\begin{tabular}{c|ccccc|c}
\toprule
Model & $MS \uparrow$ & $MP \downarrow$ & $AC \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
ConVAE & 0.052 & 0.934 & 0.250 & 0.098 & 3.115 & 0.203 \\
DiffWave & 0.000 & 0.002 & 0.001 & 0.044 & 5.027 & 0.334 \\
FACVAE & 0.061 & 0.960 & 0.206 & 0.088 & 2.936 & 0.196 \\
TCVAE & 0.129 & 1.597 & 0.236 & 0.109 & 3.244 & 0.120 \\
VDVAE & 0.066 & 1.358 & 0.112 & 0.089 & \textbf{2.856} & 0.130 \\
VDWave & 0.000 & \textbf{0.001} & 0.000 & \textbf{0.044} & 5.009 & 0.334 \\
Wavenet & 0.000 & 0.475 & 0.049 & 0.075 & 4.617 & 0.236 \\
Ours & \textbf{3.630} & 0.048 & \textbf{0.836} & 0.088 & 3.689 & \textbf{2.083} \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}[h]
\centering
\caption{Benchmark comparisons for ECG on the Mimic}
\label{tab:result_mimic_ecg}
\begin{tabular}{c|ccccc|c}
\toprule
Model & $MS \uparrow$ & $

In [25]:
# Re-load the original dataframe
SenseAccMItable = pd.read_csv('./EvalResults/SummaryTables/SenseAccMItable.csv')
SenseAccMItable['Type'] = SenseAccMItable['Type'].replace({'ART': 'ABP', 'II': 'ECG'})

# Define metric categories and mappings
major_metrics = [
    '(i) $I(V; \\acute{Z} \\mid Z)$',
    '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$',
    '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$',
    'FQI', 'RMSE', 'ISCOREScal'
]
col_map = {
    '(i) $I(V; \\acute{Z} \\mid Z)$': '(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$',
    '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$': '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$',
    '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$': '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$',
    'FQI': 'FQI $\\downarrow$',
    'RMSE': 'RMSE $\\downarrow$',
    'ISCOREScal': 'ISCORE $\\uparrow$'
}

metrics_up = [col_map[k] for k in [major_metrics[0], major_metrics[2], major_metrics[5]]]
metrics_down = [col_map[k] for k in [major_metrics[1], major_metrics[3], major_metrics[4]]]

type_name_map = {'ART': 'ABP', 'II': 'ECG'}
structural_info = ['Depth', 'LatDim', 'Comp']

# Clean model naming
SenseAccMItable.loc[SenseAccMItable['Prefix'] == 'SKZFC', 'Prefix'] = 'Ours'

# Prepare output
split_sense_tables_latex = {}

# Iterate per Source × Type
for source in SenseAccMItable['Source'].unique():
    for type_ in SenseAccMItable['Type'].unique():
        subset = SenseAccMItable[
            (SenseAccMItable['Source'] == source) &
            (SenseAccMItable['Type'] == type_)
        ]

        # Prepare and rename
        table = subset[structural_info + major_metrics].copy().reset_index(drop=True)
        table = table.rename(columns=col_map)

        # Ensure numeric for comparison
        for col in metrics_up + metrics_down:
            table[col] = pd.to_numeric(table[col], errors='coerce')

        # Format and bold
        for col in metrics_up:
            max_val = table[col].max()
            table[col] = table[col].apply(lambda x: f"\\textbf{{{x:.3f}}}" if np.isclose(x, max_val, atol=1e-8) else f"{x:.3f}")
        for col in metrics_down:
            min_val = table[col].min()
            table[col] = table[col].apply(lambda x: f"\\textbf{{{x:.3f}}}" if np.isclose(x, min_val, atol=1e-8) else f"{x:.3f}")

        table = table.rename(columns={'Depth':'$\zeta$', 'LatDim':'$J$', 'Comp':'$C$'})
        # LaTeX generation with vertical line after last structural column
        split_idx = len(structural_info)
        column_format = "c" * split_idx + "|" + "c" * (len(table.columns) - split_idx-1) +'|c'
        table = table.rename(columns={'(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$' : '$MS \\uparrow$' , 
             '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$' : '$AC \\uparrow$' , 
             '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$' : '$MP \\downarrow$'})
        
        latex_code = r"""\begin{table}[h]
\centering
\caption{Sensitivity analysis for """ + f"{type_name_map.get(type_, type_)}" + r""" on the """ + f"{source}" + r"""}
\label{tab:sense_result_""" + f"{source.lower()}_{type_.lower()}" + r"""}
""" + table.to_latex(index=False, escape=False, column_format=column_format) + r"""\end{table}"""

        split_sense_tables_latex[(source, type_)] = latex_code

# Example output
print(split_sense_tables_latex[('Mimic', 'ECG')])

\begin{table}[h]
\centering
\caption{Sensitivity analysis for ECG on the Mimic}
\label{tab:sense_result_mimic_ecg}
\begin{tabular}{ccc|ccccc|c}
\toprule
$\zeta$ & $J$ & $C$ & $MS \uparrow$ & $MP \downarrow$ & $AC \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
1 & 30 & 500 & \textbf{3.275} & 0.158 & 0.727 & 0.225 & 0.401 & \textbf{2.837} \\
1 & 30 & 800 & 2.660 & 0.202 & \textbf{0.795} & \textbf{0.223} & 0.380 & 2.442 \\
1 & 50 & 800 & 2.767 & 0.161 & 0.766 & 0.224 & 0.381 & 2.525 \\
2 & 30 & 500 & 1.321 & \textbf{0.005} & 0.501 & 1.012 & \textbf{0.380} & 1.030 \\
2 & 30 & 800 & 3.153 & 0.216 & 0.439 & 0.231 & 0.387 & 2.510 \\
2 & 50 & 800 & 3.024 & 0.251 & 0.611 & 0.294 & 0.383 & 2.437 \\
\bottomrule
\end{tabular}
\end{table}


In [26]:
print(split_sense_tables_latex[('Mimic', 'ABP')])
print()
print(split_sense_tables_latex[('Mimic', 'ECG')])
print()
print(split_sense_tables_latex[('VitalDB', 'ABP')])
print()
print(split_sense_tables_latex[('VitalDB', 'ECG')])

\begin{table}[h]
\centering
\caption{Sensitivity analysis for ABP on the Mimic}
\label{tab:sense_result_mimic_abp}
\begin{tabular}{ccc|ccccc|c}
\toprule
$\zeta$ & $J$ & $C$ & $MS \uparrow$ & $MP \downarrow$ & $AC \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
1 & 30 & 500 & \textbf{3.972} & 0.055 & \textbf{0.893} & 0.077 & 5.112 & \textbf{2.244} \\
1 & 30 & 800 & 3.630 & \textbf{0.048} & 0.836 & 0.088 & 3.689 & 2.083 \\
1 & 50 & 800 & 3.517 & 0.057 & 0.790 & 0.086 & 3.673 & 2.010 \\
2 & 30 & 500 & 3.389 & 0.143 & 0.487 & \textbf{0.066} & 4.987 & 1.772 \\
2 & 30 & 800 & 3.716 & 0.063 & 0.423 & 0.108 & 3.261 & 1.915 \\
2 & 50 & 800 & 3.603 & 0.109 & 0.345 & 0.072 & \textbf{3.259} & 1.870 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}[h]
\centering
\caption{Sensitivity analysis for ECG on the Mimic}
\label{tab:sense_result_mimic_ecg}
\begin{tabular}{ccc|ccccc|c}
\toprule
$\zeta$ & $J$ & $C$ & $MS \uparrow$ & $MP \downarrow$ & $AC \uparrow$ & FQI 

In [27]:
# Re-load the original dataframe
AblaAccMItable = pd.read_csv('./EvalResults/SummaryTables/AblaAccMItable.csv')
AblaAccMItable['Type'] = AblaAccMItable['Type'].replace({'ART': 'ABP', 'II': 'ECG'})

# Check the actual column names
actual_columns = AblaAccMItable.columns.tolist()

# Define major metrics based on actual column names
major_metrics = [
    '(i) $I(V; \\acute{Z} \\mid Z)$',
    '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$',
    '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$',
    'FQI', 'RMSE', 'ISCOREScal'
]

type_name_map = {
    'ART': 'ABP',
    'II': 'ECG'
}


# Get unique values for Source and Type columns
sources = AblaAccMItable['Source'].unique()
types = AblaAccMItable['Type'].unique()

# Rename 'SKZFC' to 'Ours' for easier interpretation
AblaAccMItable.loc[AblaAccMItable['Prefix'] == 'SKZFC', 'Prefix'] = 'Ours'
AblaAccMItable.loc[AblaAccMItable['Prefix'] == 'FC', 'Prefix'] = '$only-\\theta-prior$'
AblaAccMItable.loc[AblaAccMItable['Prefix'] == 'SKZ', 'Prefix'] = '$only-z-prior$'

# Initialize a dictionary to store the generated LaTeX tables
split_tables_latex = {}

# Generate LaTeX table for each (Source, Type) pair
for source in sources:
    for type_ in types:
        subset = AblaAccMItable[
            (AblaAccMItable['Source'] == source) & 
            (AblaAccMItable['Type'] == type_)
        ]
        
        table = subset.pivot_table(index='Prefix', values=major_metrics)
        table = table.reset_index()
        table = table[['Prefix'] + major_metrics].copy()
        
        # Rename columns
        col_map = {
            'Prefix': 'Model', 
            '(i) $I(V; \\acute{Z} \\mid Z)$': '(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$',
            '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$': '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$',
            '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$': '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$', 
            'FQI': 'FQI $\\downarrow$',
            'RMSE': 'RMSE $\\downarrow$',
            'ISCOREScal': 'ISCORE $\\uparrow$'
        }
        table = table.rename(columns=col_map)
        
        # Move Ours to last
        if 'Ours' in table['Model'].values:
            ours_row = table[table['Model'] == 'Ours']
            table = table[table['Model'] != 'Ours']
            table = pd.concat([table, ours_row], ignore_index=True)

        # Bold max values for ↑ metrics and min values for ↓ metrics
        metrics_up = ['(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$',
                      '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$',
                      'ISCORE $\\uparrow$']
        metrics_down = ['(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$',
                        'FQI $\\downarrow$', 'RMSE $\\downarrow$']

        for col in metrics_up:
            max_val = table[col].max()
            table[col] = table[col].apply(lambda x: f"\\textbf{{{x:.3f}}}" if np.isclose(x, max_val) else f"{x:.3f}")

        for col in metrics_down:
            min_val = table[col].min()
            table[col] = table[col].apply(lambda x: f"\\textbf{{{x:.3f}}}" if np.isclose(x, min_val) else f"{x:.3f}")
        table = table.rename(columns={'(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$' : '$MS \\uparrow$' , 
             '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$' : '$AC \\uparrow$' , 
             '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$' : '$MP \\downarrow$'})
        
        # Generate LaTeX
        latex_code = r"""\begin{table}[h]
\centering
\caption{Ablation studies for """ + f"{type_name_map.get(type_, type_)}" + r""" on the """ + f"{source}" + r"""}
\label{tab:result_""" + f"{source.lower()}_{type_.lower()}" + r"""}
""" + table.to_latex(index=False, escape=False, column_format="c|" + "c"*(len(table.columns)-2)+'|c' )  + r"""\end{table}"""

        split_tables_latex[(source, type_)] = latex_code


# Example: print one latex table
print(split_tables_latex[('Mimic', 'ABP')])


\begin{table}[h]
\centering
\caption{Ablation studies for ABP on the Mimic}
\label{tab:result_mimic_abp}
\begin{tabular}{c|ccccc|c}
\toprule
Model & $MS \uparrow$ & $MP \downarrow$ & $AC \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
$only-\theta-prior$ & 3.282 & 0.990 & 0.360 & 0.088 & \textbf{3.657} & 1.281 \\
$only-z-prior$ & 3.083 & \textbf{0.000} & \textbf{0.884} & \textbf{0.079} & 5.845 & 1.848 \\
Ours & \textbf{3.517} & 0.057 & 0.790 & 0.086 & 3.673 & \textbf{2.010} \\
\bottomrule
\end{tabular}
\end{table}


In [28]:
print(split_tables_latex[('Mimic', 'ABP')])
print()
print(split_tables_latex[('Mimic', 'ECG')])
print()
print(split_tables_latex[('VitalDB', 'ABP')])
print()
print(split_tables_latex[('VitalDB', 'ECG')])

\begin{table}[h]
\centering
\caption{Ablation studies for ABP on the Mimic}
\label{tab:result_mimic_abp}
\begin{tabular}{c|ccccc|c}
\toprule
Model & $MS \uparrow$ & $MP \downarrow$ & $AC \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
$only-\theta-prior$ & 3.282 & 0.990 & 0.360 & 0.088 & \textbf{3.657} & 1.281 \\
$only-z-prior$ & 3.083 & \textbf{0.000} & \textbf{0.884} & \textbf{0.079} & 5.845 & 1.848 \\
Ours & \textbf{3.517} & 0.057 & 0.790 & 0.086 & 3.673 & \textbf{2.010} \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}[h]
\centering
\caption{Ablation studies for ECG on the Mimic}
\label{tab:result_mimic_ecg}
\begin{tabular}{c|ccccc|c}
\toprule
Model & $MS \uparrow$ & $MP \downarrow$ & $AC \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
$only-\theta-prior$ & 2.851 & 0.310 & 0.672 & \textbf{0.212} & 0.389 & 2.418 \\
$only-z-prior$ & \textbf{2.923} & \textbf{0.000} & \textbf{0.853} & 0.550 & 0.443 & 2.316 \\
Ours 